In [1]:
!pip install lightgbm catboost xgboost scikit-learn --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.6 MB/s eta 0:00:00


In [2]:
from google.colab import files
uploaded = files.upload()

Saving INX_Future_Inc_Employee_Performance_CDS_Project2_Data_V1.8.csv to INX_Future_Inc_Employee_Performance_CDS_Project2_Data_V1.8.csv


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, f1_score, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
import joblib

In [4]:
df = pd.read_csv('INX_Future_Inc_Employee_Performance_CDS_Project2_Data_V1.8.csv')

In [6]:
df.shape

(1200, 28)

In [8]:
df.describe()

,Age,DistanceFromHome,EmpEducationLevel,EmpEnvironmentSatisfaction,EmpHourlyRate,EmpJobInvolvement,EmpJobLevel,EmpJobSatisfaction,NumCompaniesWorked,EmpLastSalaryHikePercent,EmpRelationshipSatisfaction,TotalWorkExperienceInYears,TrainingTimesLastYear,EmpWorkLifeBalance,ExperienceYearsAtThisCompany,ExperienceYearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,PerformanceRating
count,1200.000000,1200.000000,1200.00000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000
mean,36.918333,9.165833,2.89250,2.715833,65.981667,2.731667,2.067500,2.732500,2.665000,15.222500,2.725000,11.330000,2.785833,2.744167,7.077500,4.291667,2.194167,4.105000,2.948333
std,9.087289,8.176636,1.04412,1.090599,20.211302,0.707164,1.107836,1.100888,2.469384,3.625918,1.075642,7.797228,1.263446,0.699374,6.236899,3.613744,3.221560,3.541576,0.518866
min,18.000000,1.000000,1.00000,1.000000,30.000000,1.000000,1.000000,1.000000,0.000000,11.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,2.000000
25%,30.000000,2.000000,2.00000,2.000000,48.000000,2.000000,1.000000,2.000000,1.000000,12.000000,2.000000,6.000000,2.000000,2.000000,3.000000,2.000000,0.000000,2.000000,3.000000
50%,36.000000,7.000000,3.00000,3.000000,66.000000,3.000000,2.000000,3.000000,2.000000,14.000000,3.000000,10.000000,3.000000,3.000000,5.000000,3.000000,1.000000,3.000000,3.000000
75%,43.000000,14.000000,4.00000,4.000000,83.000000,3.000000,3.000000,4.000000,4.000000,18.000000,4.000000,15.000000,3.000000,3.000000,10.000000,7.000000,3.000000,7.000000,3.000000
max,60.000000,29.000000,5.00000,4.000000,100.000000,4.000000,5.000000,4.000000,9.000000,25.000000,4.000000,40.000000,6.000000,4.000000,40.000000,18.000000,15.000000,17.000000,4.000000


In [10]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 28 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   EmpNumber                     1200 non-null   object
 1   Age                           1200 non-null   int64 
 2   Gender                        1200 non-null   object
 3   EducationBackground           1200 non-null   object
 4   MaritalStatus                 1200 non-null   object
 5   EmpDepartment                 1200 non-null   object
 6   EmpJobRole                    1200 non-null   object
 7   BusinessTravelFrequency       1200 non-null   object
 8   DistanceFromHome              1200 non-null   int64 
 9   EmpEducationLevel             1200 non-null   int64 
 10  EmpEnvironmentSatisfaction    1200 non-null   int64 
 11  EmpHourlyRate                 1200 non-null   int64 
 12  EmpJobInvolvement             1200 non-null   int64 
 13  EmpJobLevel       

In [11]:
for col in df.select_dtypes(include='object').columns:
    df[col] = LabelEncoder().fit_transform(df[col])

In [12]:
df.head()

,EmpNumber,Age,Gender,EducationBackground,MaritalStatus,EmpDepartment,EmpJobRole,BusinessTravelFrequency,DistanceFromHome,EmpEducationLevel,...,EmpRelationshipSatisfaction,TotalWorkExperienceInYears,TrainingTimesLastYear,EmpWorkLifeBalance,ExperienceYearsAtThisCompany,ExperienceYearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition,PerformanceRating
0,0,32,1,2,2,5,13,2,10,3,...,4,10,2,2,10,7,0,8,0,3
1,1,47,1,2,2,5,13,2,14,4,...,4,20,2,3,7,7,1,7,0,3
2,2,40,1,1,1,5,13,1,5,4,...,3,20,2,3,18,13,1,12,0,4
3,3,41,1,0,0,3,8,2,10,4,...,2,23,2,2,21,6,12,6,0,3
4,4,60,1,2,2,5,13,2,16,4,...,4,10,1,3,2,2,2,2,0,3


In [18]:
original_min = df['PerformanceRating'].min()
y = df['PerformanceRating']- original_min
X = df.drop(columns=['PerformanceRating', 'EmpNumber', 'Attrition'])

In [19]:
models = {
    "RandomForest": (RandomForestClassifier(class_weight='balanced', random_state=42), {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5],
        'min_samples_leaf': [1, 2]
    }),
    "XGBoost": (XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42), {
        'n_estimators': [100, 200, 300],
        'max_depth': [3, 6, 9],
        'learning_rate': [0.01, 0.05, 0.1]
    }),
    "LightGBM": (LGBMClassifier(random_state=42), {
        'n_estimators': [100, 200, 300],
        'max_depth': [5, 10, 15],
        'learning_rate': [0.01, 0.05, 0.1]
    }),
    "CatBoost": (CatBoostClassifier(verbose=0, random_state=42), {
        'iterations': [100, 200, 300],
        'depth': [4, 6, 8],
        'learning_rate': [0.01, 0.05, 0.1]
    })
}

In [25]:
from sklearn.model_selection import cross_val_score
thresholds = [0.90, 0.95, 0.99]
results = []

for name, (model, param_grid) in models.items():
    print(f"\n{name} | Feature Selection + Grid Search")

    # Fit to extract importance
    model.fit(X, y)
    importances = model.feature_importances_
    feature_names = np.array(X.columns)
    sorted_idx = np.argsort(importances)[::-1]
    sorted_features = feature_names[sorted_idx]
    sorted_importances = importances[sorted_idx]
    cum_importance = np.cumsum(sorted_importances)

    for threshold in thresholds:
        selected_features = sorted_features[cum_importance <= threshold]
        if len(selected_features) < 1:
            selected_features = sorted_features[:1]

        X_sel = X[selected_features]

        X_train, X_test, y_train, y_test = train_test_split(
            X_sel, y, test_size=0.2, stratify=y, random_state=42
        )

        grid = GridSearchCV(model, param_grid, cv=3, scoring='f1_macro', n_jobs=-1)
        grid.fit(X_train, y_train)
        best_model = grid.best_estimator_

        y_train_pred = best_model.predict(X_train)
        y_test_pred = best_model.predict(X_test)

        y_train_true = y_train + original_min
        y_test_true = y_test + original_min
        y_train_pred_orig = y_train_pred + original_min
        y_test_pred_orig = y_test_pred + original_min

        train_f1 = f1_score(y_train_true, y_train_pred_orig, average='macro')
        test_f1 = f1_score(y_test_true, y_test_pred_orig, average='macro')
        train_acc = accuracy_score(y_train_true, y_train_pred_orig)
        test_acc = accuracy_score(y_test_true, y_test_pred_orig)

        # Cross validation score (on full selected set)
        cross_val_f1 = np.mean(cross_val_score(best_model, X_sel, y, cv=5, scoring='f1_macro'))

        # Save model
        model_filename = f"{name.lower()}_{int(threshold*100)}_modelpp.pkl"
        joblib.dump(best_model, model_filename)

        print("\nClassification Report (Test):")
        print(classification_report(y_test_true, y_test_pred_orig, digits=4))

        results.append({
            'Model': name,
            'Threshold': threshold,
            'Num Features': len(selected_features),
            'Features': list(selected_features),
            'Train F1': round(train_f1, 4),
            'Test F1': round(test_f1, 4),
            'Train Acc': round(train_acc, 4),
            'Test Acc': round(test_acc, 4),
            'CrossVal F1': round(cross_val_f1, 4),
            'Model File': model_filename
        })


RandomForest | Feature Selection + Grid Search

Classification Report (Test):
              precision    recall  f1-score   support

           2     0.9429    0.8462    0.8919        39
           3     0.9301    0.9886    0.9584       175
           4     1.0000    0.7308    0.8444        26

    accuracy                         0.9375       240
   macro avg     0.9577    0.8552    0.8983       240
weighted avg     0.9398    0.9375    0.9353       240


Classification Report (Test):
              precision    recall  f1-score   support

           2     0.9429    0.8462    0.8919        39
           3     0.9297    0.9829    0.9556       175
           4     0.9500    0.7308    0.8261        26

    accuracy                         0.9333       240
   macro avg     0.9409    0.8533    0.8912       240
weighted avg     0.9341    0.9333    0.9312       240


Classification Report (Test):
              precision    recall  f1-score   support

           2     0.9429    0.8462    0.891

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [15:30:41] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [15:30:58] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [15:30:58] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [15:30:59] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [15:30:59] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_e


Classification Report (Test):
              precision    recall  f1-score   support

           2     0.9211    0.8974    0.9091        39
           3     0.9500    0.9771    0.9634       175
           4     0.9545    0.8077    0.8750        26

    accuracy                         0.9458       240
   macro avg     0.9419    0.8941    0.9158       240
weighted avg     0.9458    0.9458    0.9450       240



/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [15:31:17] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [15:31:17] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [15:31:18] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [15:31:18] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [15:31:18] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_e


Classification Report (Test):
              precision    recall  f1-score   support

           2     0.8857    0.7949    0.8378        39
           3     0.9341    0.9714    0.9524       175
           4     0.9130    0.8077    0.8571        26

    accuracy                         0.9250       240
   macro avg     0.9109    0.8580    0.8825       240
weighted avg     0.9239    0.9250    0.9235       240



...

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Classification Report (Test):
              precision    recall  f1-score   support

           2     0.0000    0.0000    0.0000        39
           3     0.7292    1.0000    0.8434       175
           4     0.0000    0.0000    0.0000        26

    accuracy                         0.7292       240
   macro avg     0.2431    0.3333    0.2811       240
weighted avg     0.5317    0.7292    0.6150       240



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Classification Report (Test):
              precision    recall  f1-score   support

           2     0.0000    0.0000    0.0000        39
           3     0.7292    1.0000    0.8434       175
           4     0.0000    0.0000    0.0000        26

    accuracy                         0.7292       240
   macro avg     0.2431    0.3333    0.2811       240
weighted avg     0.5317    0.7292    0.6150       240



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [26]:
results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by='Test F1', ascending=False).reset_index(drop=True)

In [27]:
print("\n MODEL COMPARISON")
print(results_df[['Model', 'Threshold', 'Test F1', 'Train F1', 'Test Acc', 'Train Acc', 'CrossVal F1', 'Num Features', 'Model File']])


 MODEL COMPARISON
           Model  Threshold  Test F1  Train F1  Test Acc  Train Acc  \
0        XGBoost       0.90   0.9158    0.9548    0.9458     0.9688   
1        XGBoost       0.99   0.9052    0.9690    0.9417     0.9771   
2   RandomForest       0.99   0.8983    0.9897    0.9375     0.9917   
3   RandomForest       0.90   0.8983    0.9872    0.9375     0.9896   
4   RandomForest       0.95   0.8912    0.9835    0.9333     0.9865   
5        XGBoost       0.95   0.8825    1.0000    0.9250     1.0000   
6       LightGBM       0.90   0.2811    0.2809    0.7292     0.7281   
7       LightGBM       0.95   0.2811    0.2809    0.7292     0.7281   
8       LightGBM       0.99   0.2811    0.2809    0.7292     0.7281   
9       CatBoost       0.90   0.2811    0.2809    0.7292     0.7281   
10      CatBoost       0.95   0.2811    0.2809    0.7292     0.7281   
11      CatBoost       0.99   0.2811    0.2809    0.7292     0.7281   

    CrossVal F1  Num Features                   Model Fil

In [30]:
best = results_df.iloc[0]
print("\n BEST MODEL DETAILS")
print(f"Model: {best['Model']}")
print(f"Threshold: {best['Threshold']}")
print(f"Train F1: {best['Train F1']} | Test F1: {best['Test F1']}")
print(f"Train Accuracy: {best['Train Acc']} | Test Accuracy: {best['Test Acc']}")
print(f"Cross-Validated F1: {best['CrossVal F1']}")
print(f"Selected Features: {best['Features']}")
print(f"Saved as: {best['Model File']}")



 BEST MODEL DETAILS
Model: XGBoost
Threshold: 0.9
Train F1: 0.9548 | Test F1: 0.9158
Train Accuracy: 0.9688 | Test Accuracy: 0.9458
Cross-Validated F1: 0.8904
Selected Features: ['EmpEnvironmentSatisfaction', 'EmpLastSalaryHikePercent', 'YearsSinceLastPromotion', 'EmpDepartment', 'ExperienceYearsInCurrentRole', 'EmpWorkLifeBalance', 'OverTime', 'YearsWithCurrManager', 'TrainingTimesLastYear', 'EmpJobLevel', 'EmpJobRole', 'ExperienceYearsAtThisCompany', 'EmpJobSatisfaction', 'BusinessTravelFrequency', 'EmpRelationshipSatisfaction', 'EmpJobInvolvement']
Saved as: xgboost_90_modelpp.pkl


In [31]:
from google.colab import files
files.download('xgboost_90_modelpp.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>